In [25]:
import numpy as np
from tqdm import tqdm

### Rui's Implementation, slight modification to python, different $\hat{a}$ estimation.

In [102]:
a0, b, T = 0.00, 0.0, 100
count, iters = 0, 1000

for _ in tqdm(range(iters)):

    # generate AR(1)
    X = np.zeros(T)
    X[0] = np.random.normal(0, np.sqrt(1 / (1 - a0 ** 2)))
    
    noises = np.random.normal(0, 1, T - 1)
    for t in range(1, T):
        X[t] = b + a0 * (X[t - 1] - b) + noises[t - 1]
    
    # LOOCV parameter estimates
    hat_b = (sum(X) - X) / T
    
    hat_a = np.zeros(T)
    
    CV_0 = np.zeros(T)
    CV_1 = np.zeros(T)
    
    for t in range(2, T - 1):
        
        nominator = (X[1:] - hat_b[t]) @ (X[:-1] - hat_b[t]).T - (X[t] - hat_b[t]) * (X[t - 1] + X[t + 1] - 2 * hat_b[t])
        denominator = (sum((X[1:] - hat_b[t]) ** 2) - (X[t] - hat_b[t]) ** 2 - (X[t + 1] - hat_b[t]) ** 2)
        
        hat_a[t] = nominator / denominator
        
        CV_0[t] = (X[t]- hat_b[t] - a0       * (X[t - 1] - hat_b[t])) ** 2
        CV_1[t] = (X[t]- hat_b[t] - hat_a[t] * (X[t - 1] - hat_b[t])) ** 2
        
    count += (sum(CV_0[1:-1]) < sum(CV_1[1:-1]))

print(f"P(CV(a = a0 = {a0}) < CV(a = hat(a)) = {round(count / iters, 3)}.")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:17<00:00, 55.59it/s]

P(CV(a = a0 = 0.0) < CV(a = hat(a)) = 0.822.


### Martin's $\hat{a}$ implementation.

In [103]:
a0, b, T = 0.00, 0.8, 100
count, iters = 0, 100

for _ in tqdm(range(iters)):

    X = np.zeros(T)
    
    # generate AR(1)
    X[0] = np.random.normal(0, np.sqrt(1 / (1 - a0 ** 2)))
    
    noises = np.random.normal(0, 1, T - 1)
    for t in range(1, T):
        X[t] = b + a0 * (X[t - 1] - b) + noises[t - 1]
    
    # LOOCV parameter estimates
    hat_b = (sum(X) - X) / T
    
    hat_a = np.zeros(T)
    
    CV_0 = np.zeros(T)
    CV_1 = np.zeros(T)
    
    for t in range(T - 1):
        nominator = np.dot(X[:-1] - hat_b[t], X[1:] - hat_b[t]) - (X[t] - hat_b[t]) * (X[t + 1] - hat_b[t])
        denominator = np.dot(X[:-1] - hat_b[t], X[:-1] - hat_b[t]) - (X[t] - hat_b[t]) * (X[t] - hat_b[t])
        
        hat_a[t] = nominator / denominator
                   
        CV_0[t] = (X[t + 1]- hat_b[t] - a0       * (X[t] - hat_b[t])) ** 2
        CV_1[t] = (X[t + 1]- hat_b[t] - hat_a[t] * (X[t] - hat_b[t])) ** 2
        
    count += (sum(CV_0[:-1]) < sum(CV_1[:-1]))

print(f"P(CV(a = a0 = {a0}) < CV(a = hat(a)) = {round(count / iters, 3)}.")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.82it/s]

P(CV(a = a0 = 0.0) < CV(a = hat(a)) = 0.85.
